In [ ]:
from gurobipy import *
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from gurobipy import *
import pandas as pd

In [ ]:
m1 = Model('VRPTW')

In [ ]:


# m1 = Model('vrptw')

# # 20 customer -> 1.1 minute
# # 30 customers -> 5 minute     1.69 minutes
# # 40                           2.45 min
# # 50 customer -> 160 minute


def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw





In [ ]:
customers = 30

df = pd.read_csv("data_fin.csv")


#remove unreachable if(starting time + travelling > end )
start = 0
speed = 0.5 # 30km/hr
df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
df = df[:customers]

timeWindows = [] 
for i in range(0,customers):
    st = df['READY TIME'][i]-(df['READY TIME'][i]%10)
    et = df['DUE DATE'][i]-(df['DUE DATE'][i]%10)
    tw = [st,et]
    timeWindows.append(tw)

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i,10))
timeMatrix = []


for i in range(0,customers):
    timeMatrix.append([0]*customers)
    
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
            c= int(abs(df['XCOORD.'][i]-df['XCOORD.'][j])+abs(df['YCOORD.'][i]-df['YCOORD.'][j]))/speed
            if(c%10 == 0 or c%10 == 5):
                timeMatrix[i][j] = c
                timeMatrix[j][i] = timeMatrix[i][j]
            elif(c%10 < 5):
                timeMatrix[i][j] = c-(c%10)
                timeMatrix[j][i] = timeMatrix[i][j]
            elif(c%10 > 5):
                timeMatrix[i][j] = c+(10 - c%10)
                timeMatrix[j][i] = timeMatrix[i][j]
                
                

In [ ]:
count = 0
nodeCount = []  # assigns a number to node using count and stores its value

for i in range(0,len(discreteTimeWindows)):
    c = []
    for j in range(0, len(discreteTimeWindows[i])):
        c.append(count+1)
        count += 1
    nodeCount.append(c)

print("Node Count ", nodeCount)
print()


edgeFlows = {}
edgeNumber = 0
x = [] 
edges = {}
edgesList = []  # stores edge number going out of ith node
edgesList.append([])  # for s
# solver = pywraplp.Solver.CreateSolver('GLOP')

In [ ]:
# Graph Construction

# out of s -> assuming every node is reachable from s
e =[]
for i in range(0,len(nodeCount)):
    for j in range(0,len(nodeCount[i])):
        e.append(edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
        edges['x[%i]' % edgeNumber] = (0,nodeCount[i][j])
        edgeNumber+=1
edgesList[0]=e

for i in range(0,len(discreteTimeWindows)):
        for j in range(0,len(discreteTimeWindows[i])):    # j th node
            u = nodeCount[i][j]
            utime = discreteTimeWindows[i][j]
            e = []  # stores edges reachable from u
            for k in range(0,len(discreteTimeWindows)):
                for l in range(len(discreteTimeWindows[k])-1,-1,-1): # l th node
                    v = nodeCount[k][l]
                    vtime = discreteTimeWindows[k][l]
                    if(i!=k and utime+timeMatrix[i][k]<=vtime):
                        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
                        edges['x[%i]' % edgeNumber] = (u,v) # u -> v reachable
                        e.append(edgeNumber)
                        edgeNumber += 1
                    elif (utime+timeMatrix[i][k]>vtime):
                        break
            edgesList.append(e)

# into t
e=[]
for i in range(0,len(nodeCount)):
    for j in range(0,len(nodeCount[i])):
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
            edges['x[%i]' % edgeNumber] = (nodeCount[i][j],-1)
            #add edge number in edge list 
            edgesList[nodeCount[i][j]].append(edgeNumber)
            edgeNumber+=1
edgesList.append([])
print("Node Count after",nodeCount[-1][-1])
print()

In [ ]:
# constraints
noc = 0
# contraint for outgoing flow for a customer
  
for i in range(0,len(nodeCount)): # choose one list from nodeCount
    coeff = [] # store where coeff = 1
    for j in range(0,len(nodeCount[i])): # se
        node = nodeCount[i][j]
        for k in range(0,len(edgesList[node])):
            e = edgesList[node][k]
            coeff.append(x[e])
    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1
        

# constraint for incoming for a customer
incomingEdgeList =[]
for i in range(0,len(edgesList)):
        incomingEdgeList.append([]) # adding lists in a list
for i in edges:
        v = edges[i][1]  # u -> v
        edge_no = int(i[2:-1])
        incomingEdgeList[v].append(edge_no)

print("incoming edge list: ",incomingEdgeList)
print()

for i in range(0,len(nodeCount)):
    contraint = [0] * (edgeNumber) # indicates number of edges
    coeff = []
    for j in range(0,len(nodeCount[i])):
        node = nodeCount[i][j]
        for k in range(0,len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])

    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1

# constraint for a node incoming = outgoing
for i in range(0,len(nodeCount)):
    for j in range(0,len(nodeCount[i])):
        contraint = [0] * (edgeNumber)
        coeff = []
        node = nodeCount[i][j]
        for k in range(0,len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])
        for k in range(0,len(edgesList[node])):
            e = edgesList[node][k]
            contraint[e] = -1
            coeff.append(-x[e])
        m1.addConstr(( sum(coeff) == 0  ))
        noc+=1

In [ ]:
objectivefun = []
for i in range(0, len(edgesList[0])):
    objectivefun.append(x[edgesList[0][i]])
obj = sum(objectivefun)
m1.setObjective(obj,GRB.MINIMIZE)
                

    
m1.optimize()
print("Runtime is :",m1.Runtime)
print("objective value : ",m1.objVal)

In [ ]:
for v in m1.getVars():
    if(v.x == 1):
        print(v.varName)
#     print('%s: %g' % (v.varName,v.x))

In [ ]:
print("Number of Constraints :",noc)
# x[58]  =  1.0
# x[97]  =  1.0
# x[305]  =  1.0
# x[3116]  =  1.0
# x[5284]  =  1.0
# x[6740]  =  1.0
# x[8004]  =  1.0
# x[8503]  =  1.0
# x[9748]  =  1.0
# x[14626]  =  1.0
# x[14695]  =  1.0
# x[18536]  =  1.0
# x[18919]  =  1.0
# x[20614]  =  1.0
# x[21658]  =  1.0
# x[28427]  =  1.0
# x[30223]  =  1.0
# x[30912]  =  1.0
# x[32740]  =  1.0
# x[32998]  =  1.0
# x[33281]  =  1.0
# x[37006]  =  1.0
# x[41339]  =  1.0
# x[41724]  =  1.0
# x[46609]  =  1.0
# x[48159]  =  1.0
# x[50043]  =  1.0
# x[52062]  =  1.0
# x[52584]  =  1.0
# x[53123]  =  1.0
# x[53446]  =  1.0
# x[56483]  =  1.0
# x[57064]  =  1.0
# x[57487]  =  1.0
# x[57653]  =  1.0
# x[59751]  =  1.0
# x[60323]  =  1.0
# x[61874]  =  1.0
# x[66492]  =  1.0
# x[69145]  =  1.0
# x[69861]  =  1.0
# x[69998]  =  1.0
# x[70198]  =  1.0

In [ ]:
print(edgeNumber)

In [ ]:
# LP w edges < IP